In [65]:
from flask import Flask, request, render_template
import requests
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

app = Flask(__name__)
api_key = 'a15c794b'

In [66]:
# Function to fetch movies from OMDb API based on a search query
def fetch_movie_data(api_key, query):
    url = f'http://www.omdbapi.com/?s={query}&apikey={api_key}&type=movie'
    response = requests.get(url)
    if response.status_code == 200:
        search_results = response.json()
        if 'Search' in search_results:
            movies = []
            for movie in search_results['Search']:
                movie_details = fetch_movie_details(api_key, movie['imdbID'])
                if movie_details:
                    movies.append(movie_details)
            return movies
        else:
            print(f"No movies found for '{query}'")
            return []
    else:
        print(f"Error fetching data for '{query}': {response.status_code}")
        return []

# Function to fetch detailed movie data using IMDb ID
def fetch_movie_details(api_key, imdb_id):
    url = f'http://www.omdbapi.com/?i={imdb_id}&apikey={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data for IMDb ID '{imdb_id}': {response.status_code}")
        return None

# Function to get movie recommendations
def get_movie_recommendations(movie_title, movies_data):
    movies_data['combined_features'] = movies_data['Title'] + ' ' + movies_data['Genre']
    vectorizer = TfidfVectorizer(stop_words='english')
    feature_vectors = vectorizer.fit_transform(movies_data['combined_features'])
    similarity = cosine_similarity(feature_vectors)
    list_of_all_titles = movies_data['Title'].str.lower().tolist()
    movie_title = movie_title.lower()
    find_close_match = difflib.get_close_matches(movie_title, list_of_all_titles)
    if not find_close_match:
        print("No close matches found. Please try another movie title.")
        return
    
    close_match = find_close_match[0]
    index_of_the_movie = movies_data[movies_data.Title.str.lower() == close_match].index[0]
    similarity_score = list(enumerate(similarity[index_of_the_movie]))
    sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)

    print('Movies suggested for you:\n')
    i = 1
    for movie in sorted_similar_movies:
        index = movie[0]
        title_from_index = movies_data.iloc[index]['Title']
        if i < 30 and title_from_index.lower() != close_match:
            print(f"{i}. {title_from_index}")
            i += 1

def input_movie_title():
    movie_title = input("Enter your favourite movie title: ").strip()
    return movie_title

def main():
    movie_title = input_movie_title()
    movies = fetch_movie_data(api_key, movie_title)

    if movies:
        movies_data = pd.DataFrame(movies)
        if 'Title' not in movies_data.columns or 'Genre' not in movies_data.columns:
            print("The fetched data does not contain the required fields. Please check your API response.")
            return
        get_movie_recommendations(movie_title, movies_data)
    else:
        print("No movie data fetched. Please check your input and try again.")

if __name__ == "__main__":
    main()


Movies suggested for you:

1. The Great Gatsby in Five Minutes
2. The Great Gatsby Live Read!
3. Rocketbook Presents: The Great Gatsby
4. The Great Gatsby: Redemption
5. F. Scott Fitzgerald's 'The Great Gatsby': The Making of a Classic
